<img align="Left" alt="Dados_ao_Cubo" src="../shared_content/DADOS-AO-CUBO-Versão-04-1.png" />
<img align="right" alt="MLNet" src="../shared_content/EJ3KsfJWsAEtDaZ.png" />

## Davi Ramos -> Cientista de Dados 👋
(davi.info@gmail.com)

[![Linkedin Badge](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/davi-ramos/)](https://www.linkedin.com/in/davi-ramos/)
[![Twitter Badge](https://img.shields.io/badge/-Twitter-1DA1F2?style=flat-square&logo=Twitter&logoColor=white&link=https://twitter.com/Daviinfo/)](https://twitter.com/Daviinfo/)
<a href="https://github.com/DaviRamos"><img src="https://img.shields.io/github/followers/DaviRamos.svg?label=GitHub&style=social" alt="GitHub"></a>

# ML.Net - 

## Este exemplo irá criar um modelo de regressão linear para prever os preços das residências utilizando os seus dados de tamanho e Preço.

In [1]:
// Instalar os Pacotes do Nuget

// ML.NET
#r "nuget:Microsoft.ML"  

// ML.NET FastTree 
#r "nuget:Microsoft.ML.FastTree" 
   
//  DataFrame
#r "nuget:Microsoft.Data.Analysis"

//  XPlot
#r "nuget:XPlot.Plotly"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package XPlot.Plotly version 3.0.1

Installed package Microsoft.ML.FastTree version 1.5.2

Installed package Microsoft.ML version 1.5.2

Installed package Microsoft.Data.Analysis version 0.4.0

In [9]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using Microsoft.ML.Trainers.FastTree;
using System.IO;
using System.IO.Compression;
using System.Net.Http;
using System.Globalization;
using XPlot.Plotly;

using static Microsoft.ML.TrainCatalogBase;
using static Microsoft.ML.DataOperationsCatalog;

In [18]:
// Caminho dos arquivos
string diabetesPath = "./Diabetes/diabetes.csv";
string modelPath = @"./Diabetes.zip";

string  ConnectionStrings = @"Server=tcp:xxx.database.windows.net,1433;Initial Catalog=xxx;Persist Security Info=False;User ID=xxx;Password=xxx;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;";
  

In [19]:
//Download do Dataset
if (!File.Exists(housingPath))
{
    var contents = new HttpClient()
        .GetStringAsync("https://www.kaggle.com/saurabh00007/diabetescsv/download/MlgbaDuOXr5nOCKlz2Fn%2Fversions%2Fvv9mZKWiiUiAS9TLG0ss%2Ffiles%2Fdiabetes.csv").Result;
        
    File.WriteAllText(diabetesPath, contents);
} //  .GetStringAsync("https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv").Result;

Unhandled exception: System.IO.DirectoryNotFoundException: Could not find a part of the path '/mnt/d/Onderive/Particular/OneDrive/Projetos/ML.Net/DadosCubo_Davi/Diabetes/diabetes.csv'.
   at Interop.ThrowExceptionForIoErrno(ErrorInfo errorInfo, String path, Boolean isDirectory, Func`2 errorRewriter)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String path, OpenFlags flags, Int32 mode)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.StreamWriter.ValidateArgsAndOpenPath(String path, Boolean append, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamWriter..ctor(String path)
   at System.IO.File.WriteAllText(String path, String contents)
   at Submission#25.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 pre

In [236]:
// Formatador do Dataframe
Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 10;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

## Análise Exploratória de Dados (EDA)

In [254]:
public class Patient
{
    [LoadColumn(0)]
    public float Id { get; set; }

    [LoadColumn(1)]
    public float Pregnancies { get; set; }

    [LoadColumn(2)]
    public float Glucose { get; set; }

    [LoadColumn(3)]
    public float BloodPressure { get; set; }

    [LoadColumn(4)]
    public float SkinThickness { get; set; }

    [LoadColumn(5)]
    public float Insulin { get; set; }

    [LoadColumn(6)]
    public float BMI { get; set; }

    [LoadColumn(7)]
    public float DiabetesPedigreeFunction { get; set; }

    [LoadColumn(8)]
    public float Age { get; set; }

    [LoadColumn(9)]
    public float Output { get; set; }
}

public class DiabetesMLPrediction : Patient
{
    [ColumnName("PredictedLabel")]
    public float Prediction { get; set; }

    public float Probability { get; set; }

    public float[] Score { get; set; }
}

In [ ]:
var context = new MLContext();

var loader = context.Data.CreateDatabaseLoader<Patient>();

var connectionString = GetDbConnection();
var sqlCommand = "Select CAST(Id as REAL) as Id, Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, CAST(Output as REAL) as Output From Patient";
var dbSource = new DatabaseSource(SqlClientFactory.Instance, connectionString, sqlCommand);

Console.WriteLine("Loading data from database...");

var data = loader.Load(dbSource);
var set = context.Data.TrainTestSplit(data, testFraction: 0.2);
var trainingData = set.TrainSet;
var testData = set.TestSet;

Console.WriteLine("Preparing training operations...");
var pipeline = context.Transforms
    .Conversion.MapValueToKey(outputColumnName: "Label", inputColumnName: "Output")
    .Append(context.Transforms.Concatenate("Features", "Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"))
    .Append(context.MulticlassClassification.Trainers.OneVersusAll(
        context.BinaryClassification.Trainers.AveragedPerceptron("Label", "Features", numberOfIterations: 10))
    .Append(context.Transforms.Conversion.MapKeyToValue("PredictedLabel")));

Console.WriteLine("=============== Starting 10 fold cross validation ===============");
var crossValResults = context.MulticlassClassification.CrossValidate(data: trainingData, estimator: pipeline, numberOfFolds: 10, labelColumnName: "Label");
var metricsInMultipleFolds = crossValResults.Select(r => r.Metrics);
var microAccuracyValues = metricsInMultipleFolds.Select(m => m.MicroAccuracy);
var microAccuracyAverage = microAccuracyValues.Average();
var macroAccuracyValues = metricsInMultipleFolds.Select(m => m.MacroAccuracy);
var macroAccuracyAverage = macroAccuracyValues.Average();
var logLossValues = metricsInMultipleFolds.Select(m => m.LogLoss);
var logLossAverage = logLossValues.Average();
var logLossReductionValues = metricsInMultipleFolds.Select(m => m.LogLossReduction);
var logLossReductionAverage = logLossReductionValues.Average(); Console.WriteLine($"*************************************************************************************************************");
Console.WriteLine($"*       Metrics Multi-class Classification model      ");
Console.WriteLine($"*------------------------------------------------------------------------------------------------------------");
Console.WriteLine($"*       Average MicroAccuracy:    {microAccuracyAverage:0.###} ");
Console.WriteLine($"*       Average MacroAccuracy:    {macroAccuracyAverage:0.###} ");
Console.WriteLine($"*       Average LogLoss:          {logLossAverage:#.###} ");
Console.WriteLine($"*       Average LogLossReduction: {logLossReductionAverage:#.###} ");
Console.WriteLine($"*************************************************************************************************************");

//Now we need to train the model using below code    
Console.WriteLine($"Training process is starting. {DateTime.Now.ToLongTimeString()}");
var model = pipeline.Fit(trainingData);
Console.WriteLine($"Training process has finished. {DateTime.Now.ToLongTimeString()}");

//

var testPatients = context.Data.CreateEnumerable<Patient>(testData, reuseRowObject: true);
Console.WriteLine($"Test Set: {testPatients.Count()} patients");

var predictionEngine = context.Model.CreatePredictionEngine<Patient, DiabetesMLPrediction>(model);

var patient = new Patient()
{
    Age = 50,
    BloodPressure = 72,
    BMI = 33.6f,
    DiabetesPedigreeFunction = 0.627f,
    Glucose = 148,
    Insulin = 0,
    Pregnancies = 6,
    SkinThickness = 35,
    Id = 0,
    Output = 0
};

var prediction = predictionEngine.Predict(patient);
Console.WriteLine($"Diabetes? {prediction.Output} | Prediction: {(Convert.ToBoolean(prediction.Prediction) ? "Yes" : "No")} | Probability: {prediction.Probability} ");

Console.WriteLine("Saving the model");
context.Model.Save(model, trainingData.Schema, "MLModel.zip");

In [255]:
// Criar o MLContext para ser compartilhado entre os objetos de fluxo de trabalho de criação do modelo
// Definir uma semente aleatória para resultados repetíveis/determinísticos nos vários treinamentos.
MLContext mlContext = new MLContext(seed: 1);

In [256]:
// Métricas de Avalição do Modelo
public static void PrintRegressionMetrics(RegressionMetrics metrics)
{
    Console.WriteLine($"*************************************************");
    Console.WriteLine($"*       Metrics for Regression model      ");
    Console.WriteLine($"*------------------------------------------------");
    Console.WriteLine($"*       LossFn:        {metrics.LossFunction:0.##}");
    Console.WriteLine($"*       R2 Score:      {metrics.RSquared:0.##}");
    Console.WriteLine($"*       Absolute loss: {metrics.MeanAbsoluteError:#.##}");
    Console.WriteLine($"*       Squared loss:  {metrics.MeanSquaredError:#.##}");
    Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
    Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
    Console.WriteLine($"*************************************************");
}

public static void PrintRegressionFoldsAverageMetrics(IEnumerable<TrainCatalogBase.CrossValidationResult<RegressionMetrics>> crossValidationResults)
{
    var L1 = crossValidationResults.Select(r => r.Metrics.MeanAbsoluteError);
    var L2 = crossValidationResults.Select(r => r.Metrics.MeanSquaredError);
    var RMS = crossValidationResults.Select(r => r.Metrics.RootMeanSquaredError);
    var lossFunction = crossValidationResults.Select(r => r.Metrics.LossFunction);
    var R2 = crossValidationResults.Select(r => r.Metrics.RSquared);
    
    Console.WriteLine($"*************************************************************************************************************");
    Console.WriteLine($"*       Metrics for Regression model      ");
    Console.WriteLine($"*------------------------------------------------------------------------------------------------------------");
    Console.WriteLine($"*       Average L1 Loss:       {L1.Average():0.###} ");
    Console.WriteLine($"*       Average L2 Loss:       {L2.Average():0.###}  ");
    Console.WriteLine($"*       Average RMS:           {RMS.Average():0.###}  ");
    Console.WriteLine($"*       Average Loss Function: {lossFunction.Average():0.###}  ");
    Console.WriteLine($"*       Average R-squared:     {R2.Average():0.###}  ");    
    Console.WriteLine($"*************************************************************************************************************");
}

In [257]:
private static IDataView trainingDataView;
private static IDataView testingDataView;

//Configuração do carregamento de dados         
IDataView fullData = mlContext.Data.LoadFromTextFile<ModelInput>(path: housingPath, hasHeader: true, 
                                                                         separatorChar: ',', allowQuoting: true, allowSparse: false);

// Divide o conjunto de dados em duas partes: TrainingDataset (80%) e TestDataset (20%)
DataOperationsCatalog.TrainTestData trainTestData = mlContext.Data.TrainTestSplit(fullData, testFraction: 0.2);
trainingDataView = trainTestData.TrainSet;
testingDataView = trainTestData.TestSet;

In [258]:
// Criar e treinar o modelo
// Configuração do processo de Pipeline de transformação de dados 
var dataProcessPipeline = mlContext.Transforms.Categorical.OneHotEncoding(new[] 
                                                { new InputOutputColumnPair("Ocean_proximity", "Ocean_proximity") })
                                                  .Append(mlContext.Transforms.Concatenate("Features", new[] { "Ocean_proximity", 
                                                                                        "Longitude", "Latitude", "Housing_median_age", 
                                                                                        "Total_rooms", "Total_bedrooms", "Population", 
                                                                                                        "Households", "Median_income" })
                                                 .AppendCacheCheckpoint(mlContext));
// Definir o algoritmo de treinamento
var trainer = mlContext.Regression.Trainers.Sdca(labelColumnName: "Label", featureColumnName: "Features");

var trainingPipeline = dataProcessPipeline.Append(trainer);


(3,27): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'Microsoft.DotNet.Interactive.Formatting' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=3.1.7.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



In [259]:
Console.WriteLine("=============== Treinando o modelo ===============");

var model = trainingPipeline.Fit(trainingDataView);

Console.WriteLine("=============== Fim do Processo de Treinamento ===============");

=============== Treinando o modelo ===============
=============== Fim do Processo de Treinamento ===============


In [260]:
// Avalie a precisão do modelo
IDataView predictions = model.Transform(testingDataView);
Console.WriteLine("===== Avaliando a acurácia do modelo com dados de teste =====");
var metrics = mlContext.Regression.Evaluate(data:predictions, labelColumnName:"Label", scoreColumnName: "Score"); 
PrintRegressionMetrics(metrics);

===== Avaliando a acurácia do modelo com dados de teste =====
*************************************************
*       Metrics for Regression model      
*------------------------------------------------
*       LossFn:        8451832124.8
*       R2 Score:      0.37
*       Absolute loss: 67978.71
*       Squared loss:  8451832146.09
*       RMS loss:      91933.85
*       RMS loss:      91933.85
*************************************************


In [261]:
// Salvar/persistir o modelo treinado em um arquivo .ZIP
Console.WriteLine($"=============== Salvando o modelo  ===============");
mlContext.Model.Save(model, trainingDataView.Schema, modelPath); 
Console.WriteLine("O Modelo foi salvo em {0}", modelPath);

=============== Salvando o modelo  ===============
O Modelo foi salvo em ./HouseModelv2.zip


In [262]:
Console.WriteLine("=============== Previsão para um único caso  ===============");

ModelInput sampleData = new ModelInput()
{
    Longitude = -122.23F,
    Latitude = 37.88F,
    Housing_median_age = 41F,
    Total_rooms = 880F,
    Total_bedrooms = 129F,
    Population = 322F,
    Households = 126F,
    Median_income = 8.3252F,
    Ocean_proximity = @"NEAR BAY",
};

=============== Previsão para um único caso  ===============


In [263]:
ITransformer model = mlContext.Model.Load(modelPath, out var modelInputSchema);
// Criar um mecanismo de predição relacionado ao modelo treinado carregado
var predEngine = mlContext.Model.CreatePredictionEngine <ModelInput, ModelOutput>(model);

//Score
var resultprediction = predEngine.Predict (sampleData);

Console.WriteLine("Usando o modelo para fazer predição única - Comparando o valor atual de Median_house_value com o valor previsto de para os dados de amostra de Median_house_value ... \n \n");
Console.WriteLine($"Longitude: {sampleData.Longitude}");
Console.WriteLine($"Latitude: {sampleData.Latitude}");
Console.WriteLine($"Housing_median_age: {sampleData.Housing_median_age}");
Console.WriteLine($"Total_rooms: {sampleData.Total_rooms}");
Console.WriteLine($"Total_bedrooms: {sampleData.Total_bedrooms}");
Console.WriteLine($"Population: {sampleData.Population}");
Console.WriteLine($"Households: {sampleData.Households}");
Console.WriteLine($"Median_income: {sampleData.Median_income}");
Console.WriteLine($"Ocean_proximity: {sampleData.Ocean_proximity}");
Console.WriteLine ($"\n \nO Preço previsto para este tamanho é: {resultprediction.Score} \n \n");
Console.WriteLine ("=============== Fim do processo ===============");

Usando o modelo para fazer predição única - Comparando o valor atual de Median_house_value com o valor previsto de para os dados de amostra de Median_house_value ... 
 

Longitude: -122.23
Latitude: 37.88
Housing_median_age: 41
Total_rooms: 880
Total_bedrooms: 129
Population: 322
Households: 126
Median_income: 8.3252
Ocean_proximity: NEAR BAY

 
O Preço previsto para este tamanho é: 308464.28 
 

=============== Fim do processo ===============


## Fluxo de trabalho de código

Neste Artigo  percorremos todas as etapas abaixo
<ul>
   <li>Coletar e carregar dados de treinamento em um objeto IDataView</li>
   <li>Especifique um pipeline de operações para extrair recursos e aplicar um algoritmo de aprendizado de máquina</li>
   <li>Treinar um modelo chamando Fit() no pipeline</li>
   <li>Avaliar o modelo e iterar para melhorar</li>
   <li>Salvar o modelo em um formato binário para uso em um aplicativo</li>
   <li>Carregar o modelo de volta para um objeto ITransformer</li>
   <li>Fazer previsões chamando CreatePredictionEngine.Predict()</li>
</ul>

![Fluxo](../shared_content/mldotnet-annotated-workflow.png)

# Em nosso próximo artigo veremos como implantar o modelo criado numa api Web para consumo por outra aplicações.